# Generacion de texto con GPT2 de KerasNLP

**Author:** David Alonso Quispe Castillo<br>
**Date created:** 14/07/2023<br>
**Last modified:** 17/08/2023<br>
**Objetivo:** Usar el modelo GPT-2 de KerasNLP y los generadores de texto (samplers) para la generación de texto.

En este proyecto, basado en el trabajo previo de [Chen Qian](https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/generative/ipynb/gpt2_text_generation_with_kerasnlp.ipynb#scrollTo=76gl9HSIF-uB), utilizaremos [KerasNLP](https://keras.io/keras_nlp/) para cargar un modelo de lenguaje grande  (LLM) pre-entrenado - el modelo [GPT-2 ](https://openai.com/research/better-language-models), desarrollado por OpenAI. Aprovecharemos las capacidades de generación de texto de este modelo para crear texto basado en una entrada proporcionada por el usuario. Además, se demostrará cómo GPT-2 puede adaptarse rápidamente a otros idiomas, como el español.

In [1]:
import shutil
shutil.rmtree("/kaggle/working/")

OSError: [Errno 16] Device or resource busy: '/kaggle/working/'

https://www.tensorflow.org/install/source?hl=es-419#gpu

##  Antes de comenzar (Colab)

Colab ofrece diferentes tipos de entornos de ejecución. Asegúrate de ir a **Entorno de ejecución -> Cambiar tipo de entorno de ejecución** y seleccionar el entorno de ejecución con aceleración de hardware GPU (que debería tener >12 GB de RAM del host y ~15 GB de RAM de la GPU) ya que ajustaremos finamente el modelo GPT-2. Ejecutar este tutorial en un entorno de ejecución de CPU tomaría horas.

en colab las versiones usadas son:



Versión de keras-nlp: 0.6.0

Versión de TensorFlow: 2.13.0

Versión de Python: 3.10.6 (main, May 29 2023, 11:10:38) [GCC 11.3.0]

Versión de Cudnn:
define CUDNN_MAJOR 8
define CUDNN_MINOR 9
define CUDNN_PATCHLEVEL 0
define CUDNN_VERSION (CUDNN_MAJOR * 1000 + CUDNN_MINOR * 100 + CUDNN_PATCHLEVEL)

/* cannot use constexpr here since this is a C-only file */

Versión de CUDA :
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0

Dispositivo GPU : /physical_device:GPU:0

Escpecificaciones de gpu:
Wed Jul 19 21:24:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU   GI   CI        PID   Type   Process name                  GPU Memory |
|        ID   ID                                                   Usage      |
|=============================================================================|
|  No running processes found                                                 |
+-----------------------------------------------------------------------------+


##  Antes de comenzar (en Amazon Sagemaker Studio lab)

Se optó por usar estas versiones para la ejecución:
- Python: 3.9.15
- Tensorflow: 2.11.0
- keras-nlp: 0.6.0

La razon de usar dichas versiones es porque sage maker studio lab, viene por defecto con:
- CUDA: 11.2
- cudnn: 8.1


Si solamente instalas tensorflow en tu conda enviroment, y quieres ejecutar la funcion generate() del modelo te aparecerá un error de dependecia por parte del paquete tensorflow-text, y si intentas realizar la instalacion de la version correcta mediante el siguiente codigo: pip install tensorflow-text~=2.11.0, no se instalará esa version, sino la mas reciente que es la 2.13.0 y ademas que instalará toda la version completa de tensorflow 2.13.0, para instalar la version necesaria que es la 2.11.0 debes ejecutar lo siguiente:

pip install tensorflow==2.11.0
pip install h5py
pip install typing-extensions
pip install wheel

y al final:

pip install tensorflow-text~=2.11.0

instalar desde ya ipywidgets sino en algun momento saldra error, no recuerdo cuando dx
pip install ipywidgets

instalar esta version de protobuf porque habia error al momento de entrenar el modelo:
pip install protobuf~=3.20.0

(hasta el momento esta entrenando en el primer epoch, pero esta mostrando estos alertas de error:)


ERROR:absl:module 'tensorflow_datasets.core' has no attribute 'utils'
Traceback (most recent call last):
  File "/home/studio-lab-user/.conda/envs/gpt/lib/python3.9/site-packages/tensorflow_datasets/__init__.py", line 59, in <module>
    from tensorflow_datasets import audio
  File "/home/studio-lab-user/.conda/envs/gpt/lib/python3.9/site-packages/tensorflow_datasets/audio/__init__.py", line 19, in <module>
    from tensorflow_datasets.audio.commonvoice import CommonVoice
  File "/home/studio-lab-user/.conda/envs/gpt/lib/python3.9/site-packages/tensorflow_datasets/audio/commonvoice.py", line 27, in <module>
    import tensorflow_datasets.public_api as tfds
  File "/home/studio-lab-user/.conda/envs/gpt/lib/python3.9/site-packages/tensorflow_datasets/public_api.py", line 57, in <module>
    deprecated = core.utils.docs.deprecated(deprecated)
AttributeError: module 'tensorflow_datasets.core' has no attribute 'utils'
Epoch 1/10
2023-07-19 22:04:00.467755: W tensorflow/compiler/tf2xla/kernels/random_ops.cc:57] Warning: Using tf.random.uniform with XLA compilation will ignore seeds; consider using tf.random.stateless_uniform instead if reproducible behavior is desired. gpt2_causal_lm/gpt2_backbone/embeddings_dropout/dropout/random_uniform/RandomUniform
2023-07-19 22:04:01.556077: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:38] Ignoring Assert operator sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
 682/2425 [=======>......................] - ETA: 13:11 - loss: 3.4543 - sparse_categorical_accuracy: 0.3821

## Introducción a los Modelos de Lenguaje Generativos de Gran Escala (LLMs)

Los Modelos de Lenguaje Grande (LLMs, por sus siglas en inglés) son un tipo de modelos de aprendizaje automático que se entrenan en un gran corpus de datos de texto para generar resultados en varias tareas de Procesamiento del Lenguaje Natural (NLP, por sus siglas en inglés), como generación de texto, respuesta a preguntas o traducción automática.

Los LLMs generativos se basan típicamente en redes neuronales de aprendizaje profundo, como la arquitectura [Transformer](https://arxiv.org/abs/1706.03762) inventada por investigadores de Google en 2017, y se entrenan con grandes cantidades de datos de texto, a menudo involucrando miles de millones de palabras. Estos modelos, como [LaMDA](https://blog.google/technology/ai/lamda/) y [PaLM](https://ai.googleblog.com/2022/04/pathways-language-model-palm-scaling-to.html) de Google, se entrenan con conjuntos de datos extensos provenientes de diversas fuentes de datos, lo que les permite generar resultados para múltiples tareas. El núcleo de los LLMs generativos consiste en predecir la siguiente palabra en una oración, a menudo conocido como **Preentrenamiento LM Causal**. De esta manera, los LLMs pueden generar texto coherente basado en las indicaciones del usuario. Para obtener una discusión más pedagógica sobre los modelos de lenguaje, puedes consultar la [clase de Stanford CS324 LLM](https://stanford-cs324.github.io/winter2022/lectures/introduction/).

## Instalamos KerasNLP e Importamos Dependencias

In [1]:
!pip install -q keras-nlp

In [2]:
import keras_nlp
import tensorflow as tf
from tensorflow import keras
import time
import sys

print("Versión de keras-nlp:", keras_nlp.__version__)
print("\nVersión de TensorFlow:", tf.__version__)
print("\nVersión de Python:", sys.version)

print("\nVersión de Cudnn:")
!cat /usr/include/x86_64-linux-gnu/cudnn_v*.h | grep CUDNN_MAJOR -A 2
!cat /usr/include/cudnn.h | grep CUDNN_MAJOR -A 2


print("\nVersión de CUDA :")
!nvcc --version
print("\nDispositivo GPU :", tf.config.list_physical_devices('GPU')[0].name)
print("\nEscpecificaciones de gpu:")
!nvidia-smi

Using TensorFlow backend


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5

  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']

caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']

  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open fi

Versión de keras-nlp: 0.6.1



Versión de TensorFlow: 2.12.0



Versión de Python: 3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:40:32) [GCC 12.3.0]



Versión de Cudnn:

#define CUDNN_MAJOR 8

#define CUDNN_MINOR 9

#define CUDNN_PATCHLEVEL 0

--

#define CUDNN_VERSION (CUDNN_MAJOR * 1000 + CUDNN_MINOR * 100 + CUDNN_PATCHLEVEL)



/* cannot use constexpr here since this is a C-only file */



Versión de CUDA :

nvcc: NVIDIA (R) Cuda compiler driver

Copyright (c) 2005-2022 NVIDIA Corporation

Built on Wed_Sep_21_10:33:58_PDT_2022

Cuda compilation tools, release 11.8, V11.8.89

Build cuda_11.8.r11.8/compiler.31833905_0



Dispositivo GPU : /physical_device:GPU:0



Escpecificaciones de gpu:

Sun Aug  6 00:20:37 2023       

+-----------------------------------------------------------------------------+

| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |

|-------------------------------+----------------------+----------------------+

| GPU  Na

## Introducción a KerasNLP

La construcción y entrenamiento de Modelos de Lenguaje Grande (LLMs) desde cero son complejos y costosos. Afortunadamente, existen LLMs pre-entrenados disponibles para su uso inmediato. [KerasNLP](https://keras.io/keras_nlp/) proporciona una amplia variedad de puntos de control pre-entrenados que te permiten experimentar con modelos de última generación sin necesidad de entrenarlos desde cero.

KerasNLP es una biblioteca de procesamiento del lenguaje natural que brinda soporte a los usuarios a lo largo de todo su ciclo de desarrollo. KerasNLP ofrece tanto modelos pre-entrenados como componentes modulares, lo que permite a los desarrolladores reutilizar fácilmente los modelos pre-entrenados o construir sus propios LLM.

En pocas palabras, para los LLMs generativos, KerasNLP ofrece:

- Modelos pre-entrenados con el método `generate()`, por ejemplo en, `keras_nlp.models.GPT2CausalLM` y `keras_nlp.models.OPTCausalLM`.
- Clase de muestreo (Sampler) que implementa algoritmos de generación como Top-K, Beam y búsqueda contrastiva. Estos samplers se pueden utilizar para generar texto con modelos personalizados.



## Cargamos un modelo pre-entrenado de GPT-2 y generamos texto

KerasNLP proporciona varios modelos pre-entrenados, como [Google
Bert](https://ai.googleblog.com/2018/11/open-sourcing-bert-state-of-art-pre.html)
y [GPT-2](https://openai.com/research/better-language-models). Puedes ver
la lista de modelos disponibles en el [repositorio de KerasNLP](https://github.com/keras-team/keras-nlp/tree/master/keras_nlp/models).

Es muy fácil cargar el modelo GPT-2, como puedes ver a continuación:

In [3]:
# Para acelerar el entrenamiento y la generación, utilizamos un preprocesador
# de longitud 128 en lugar de la longitud completa de 1024.

preprocessor = keras_nlp.models.GPT2CausalLMPreprocessor.from_preset(
    "gpt2_base_en",
    sequence_length=128,
)
gpt2_lm = keras_nlp.models.GPT2CausalLM.from_preset(
    "gpt2_base_en", preprocessor=preprocessor
)


1042301/1042301 [==============================] - 0s 0us/step


456318/456318 [==============================] - 0s 0us/step


497986112/497986112 [==============================] - 3s 0us/step


Una vez que el modelo está cargado, puedes usarlo para generar texto de inmediato. Ejecuta las celdas a continuación para probarlo. Es tan simple como llamar a la función generate() una sola vez.

In [4]:
start = time.time()

output = gpt2_lm.generate("My trip to Yosemite was", max_length=200)
print("\nGPT-2 output:")
print(output)

end = time.time()
print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")



GPT-2 output:

My trip to Yosemite was a little over two years ago. I spent the last three years in Yosemite National Park, hiking and camping, visiting many great places in the region. I was fortunate to meet and enjoy a number of great local attractions including the Yosemite National Park, the Yosemite National Park Zoo, the Yosemite National Park Zoo, Yosemite National Park's Grand Canyon, Yosemite National Park Zoo, Yosemite National Park's Yosemite National Park Zoo, and Yosemite's Great White Shark. The last time I visited Yosemite I went with my family, but it wasn't until I went to Yosemite's Big Blue Lodge that I realized I could enjoy the beauty of the park.



My experience with Yosemite is a little more personal, but I hope that I've shown you that there's a lot to enjoy in Yosemite. I love Yosemite because it is a place of true adventure. It has a beautiful view of the entire region. I also like the views that the Grand Canyon has. I love the view of

TOTAL TIME ELAPSED

Intenta con otro ejemplo:

In [5]:
start = time.time()

output = gpt2_lm.generate("That Italian restaurant is", max_length=200)
print("\nGPT-2 output:")
print(output)

end = time.time()
print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")



GPT-2 output:

That Italian restaurant is now closed. The owner says it's because he doesn't have any money to pay staff. He says it was just because he was a "bad person" and that his employees are "a bunch of idiots who are not paying their bills."



But the restaurateur says that's not the case.



"I am not a person who wants to be a bad person," he said. "It is my right. If someone wants to be a bad person, I am the person."



The restaurant's manager, who asked not to be named because he's not authorized to speak to the media, says the restaurant will be closing.



"It's just a sad story," she said. "We have a lot to do to help people. We have a great restaurant and we're very proud to have a great customer base."



The restaurant was open for about two weeks last week, according to the restaurant's website. But after

TOTAL TIME ELAPSED: 1.77s


Observa cuán más rápido es el segundo llamado. Esto se debe a que el grafo computacional se compila con [XLA](https://www.tensorflow.org/xla) en la primera ejecución y se reutiliza en la segunda, en segundo plano.

La calidad del texto generado parece estar bien, pero podemos mejorarlo mediante el ajuste fino / afinación (fine-tuning.).

## Más sobre el modelo GPT-2 de KerasNLP

A continuación, vamos a ajustar finamente el modelo para actualizar sus parámetros, pero antes de hacerlo, echemos un vistazo al conjunto completo de herramientas que tenemos para trabajar con GPT2.

El código de GPT2 se puede encontrar [aquí](https://github.com/keras-team/keras-nlp/blob/master/keras_nlp/models/gpt2/). Conceptualmente, `GPT2CausalLM` se puede descomponer jerárquicamente en varios módulos en KerasNLP, todos los cuales tienen una función *from_preset()* que carga un modelo pre-entrenado:

- `keras_nlp.models.GPT2Tokenizer`: El tokenizador utilizado por el modelo GPT2, que es un [codificador de pares de bytes](https://huggingface.co/course/chapter6/5?fw=pt).
- `keras_nlp.models.GPT2CausalLMPreprocessor`: el preprocesador utilizado en el entrenamiento causal de GPT2. Realiza la tokenización junto con otros trabajos de preprocesamiento, como la creación de la etiqueta y la adición del token de finalización.
- `keras_nlp.models.GPT2Backbone`: el modelo GPT2, que es una pila de `keras_nlp.layers.TransformerDecoder`. A esto generalmente se le denomina simplemente `GPT2`.
- `keras_nlp.models.GPT2CausalLM`: envuelve a `GPT2Backbone`, multiplica la salida de `GPT2Backbone` por la matriz de embedding para generar logits sobre los tokens del vocabulario.

## Ajuste fino (Fine-tuning) en el conjunto de datos WikiHow

Ahora que tienes conocimientos sobre el modelo GPT-2 de KerasNLP, puedes dar un paso más para ajustar finamente el modelo y lograr que genere texto en un estilo específico, ya sea corto o largo, formal o casual. En este tutorial, utilizaremos el conjunto de datos de Wikihow como ejemplo.



## Fine-tuning con el Dataset en Español

También podemos ajustar finamente GPT2 en conjuntos de datos que no sean su idioma principal de entrenamiento. Esta parte muestra cómo ajustar finamente GPT2 en un conjunto de datos de wikihow en español para enseñar a nuestro modelo a convertirse en un sabelotodo.

Debido a que GPT2 utiliza un codificador de pares de bytes (byte-pair encoder) y el conjunto de datos de pre-entrenamiento original contiene algunos caracteres de gran parte de idiomas, podemos utilizar el vocabulario original para ajustar finamente en un conjunto de datos en español.

In [6]:
!# Descargamos el dataset desde HuggingFace
!git clone https://huggingface.co/datasets/daqc/wikihow-spanish

Cloning into 'wikihow-spanish'...

remote: Enumerating objects: 14, done.

remote: Total 14 (delta 0), reused 0 (delta 0), pack-reused 14

Unpacking objects: 100% (14/14), 2.54 KiB | 651.00 KiB/s, done.


Echemos un vistazo dentro de los datos de muestra del conjunto de datos:


In [7]:
import json

# Leer el archivo JSON
with open('wikihow-spanish/spanish.json', 'r') as file:
    data = json.load(file)

# Obtener la primera clave y el primer elemento del diccionario
primer_clave = next(iter(data))
primer_elemento = data[primer_clave]

# Imprimir la clave y la estructura del primer elemento
print("Clave:", primer_clave)
print("Estructura:")
print(json.dumps(primer_elemento, indent=4))

Clave: https://es.wikihow.com/calcular-el-rendimiento-anualizado-de-una-cartera-de-inversiones

Estructura:

{

    "Calcular tu rendimiento anualizado": {

        "summary": "Calcula tu rendimiento anualizado. Calcula el rendimiento semestral. Calcula un equivalente anualizado.",

        "document": "Una vez que hayas calculado el rendimiento total (como se muestra arriba), ingresa el resultado en esta ecuaci\u00f3n: rendimiento anualizado = (1+ rendimiento)1/N-1 El producto de esta ecuaci\u00f3n ser\u00e1 el n\u00famero correspondiente al rendimiento de cada a\u00f1o durante todo el periodo de tiempo.  En el exponente (el n\u00famero peque\u00f1o que est\u00e1 afuera del par\u00e9ntesis), el \u201c1\u201d representa la unidad que estamos midiendo, que es un a\u00f1o. Si deseas ser m\u00e1s espec\u00edfico, podr\u00edas usar \u201c365\u201d para obtener el rendimiento diario. La \u201cN\u201d representa el n\u00famero de periodos que medir\u00e1s. Entonces, si mides tu rendimiento e

En nuestro caso, al realizar la predicción de la siguiente palabra en un modelo de lenguaje, necesitamos obtener solamente la informacion de relevancia del dataset mediante el siguiente codigo:

In [8]:
import os
import json
from urllib.parse import urlsplit, unquote

wikihow_data = []
for file in os.listdir("wikihow-spanish/"):
    if ".json" not in file:
        continue
    full_filename = os.path.join("wikihow-spanish/", file)
    with open(full_filename, "r") as f:
        content = json.load(f)
        wikihow_data.append(content)

paragraphs = []
for parent_url, sections in data.items():
    decoded_parent_url = "¿Cómo " + unquote(urlsplit(parent_url).path.split("/")[-1]).replace('-', ' ') + "?"
    paragraph = f"{decoded_parent_url}\n"
    for section in sections:
        section_data = sections[section]
        title = section + ":"
        summary = section_data.get("summary", "")
        document = section_data.get("document", "")
        paragraph += f"{title}\n{summary}\n{document}\n"
    paragraphs.append(paragraph)

Damos un vistazo a los datos de ejemplo.

In [9]:
print("Parrafo 1:")
print(paragraphs[0])
#print("Parrafo 2:")
#print(paragraphs[1])

Parrafo 1:

¿Cómo calcular el rendimiento anualizado de una cartera de inversiones?

Calcular tu rendimiento anualizado:

Calcula tu rendimiento anualizado. Calcula el rendimiento semestral. Calcula un equivalente anualizado.

Una vez que hayas calculado el rendimiento total (como se muestra arriba), ingresa el resultado en esta ecuación: rendimiento anualizado = (1+ rendimiento)1/N-1 El producto de esta ecuación será el número correspondiente al rendimiento de cada año durante todo el periodo de tiempo.  En el exponente (el número pequeño que está afuera del paréntesis), el “1” representa la unidad que estamos midiendo, que es un año. Si deseas ser más específico, podrías usar “365” para obtener el rendimiento diario. La “N” representa el número de periodos que medirás. Entonces, si mides tu rendimiento en 7 años, tendrás que utilizar este número en lugar de "N". Por ejemplo, supongamos que en un periodo de siete años, el valor de tu cartera creció de $1,000 a $2,500. Primero calcula 

Ahora puedes realizar el proceso de fine-tuning del modelo usando la función *fit()* que ya conoces. Ten en cuenta que el `preprocessor` se llamará automáticamente dentro del método `fit` ya que `GPT2CausalLM` es una instancia de `keras_nlp.models.Task`.

Este paso requiere bastante memoria de la GPU y mucho tiempo si quisiéramos entrenarlo hasta alcanzar un estado completamente entrenado. En este caso, solo utilizamos una parte del conjunto de datos con fines de demostración.

In [10]:
import tensorflow_datasets as tfds
import tensorflow.keras as keras
import os
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow.keras.callbacks as callbacks

# Preparar el conjunto de datos
train_ds = (
    tf.data.Dataset.from_tensor_slices(paragraphs)
    .batch(16)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)

# Definir el número de épocas a 1 para demostración
num_epochs = 10
# Recorrer todo el conjunto de datos lleva mucho tiempo, solo tomemos `500`
# train_ds = train_ds.take(500)


# Calcular el tamaño para el conjunto de validación (20% del dataset de entrenamiento)
val_size = int(0.2 * len(paragraphs))

# Dividir el conjunto de datos de entrenamiento en entrenamiento y validación
val_ds = train_ds.take(val_size)


learning_rate = keras.optimizers.schedules.PolynomialDecay(
    5e-4,
    decay_steps=train_ds.cardinality() * num_epochs,
    end_learning_rate=0.0,
)

loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

gpt2_lm.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=loss,
    weighted_metrics=["accuracy"],  # Agregar 'mean_squared_error' y cualquier otra métrica que desees
)

# Directorio para guardar los checkpoints
checkpoint_dir = "checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

# Callback para guardar checkpoints por epoch
checkpoint_callback = ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, "model_{epoch:02d}.ckpt"),
    save_freq="epoch",
    save_weights_only=True,
    save_best_only=False,
    verbose=1
)


# Callback personalizado para obtener el tiempo por época y tiempo acumulado
class TimeHistory(callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []
        self.start_time = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.start_time)
        self.start_time = time.time()

# Crear el callback para obtener el tiempo por época y tiempo acumulado
time_callback = TimeHistory()

# Entrenar el modelo
history = gpt2_lm.fit(train_ds, epochs=num_epochs, validation_data=val_ds, callbacks=[checkpoint_callback, time_callback])


Epoch 1/10

2425/2425 [==============================] - ETA: 0s - loss: 2.3361 - accuracy: 0.5358

Epoch 1: saving model to checkpoints/model_01.ckpt

2425/2425 [==============================] - 1849s 736ms/step - loss: 2.3361 - accuracy: 0.5358 - val_loss: 1.8612 - val_accuracy: 0.6025

Epoch 2/10

2425/2425 [==============================] - ETA: 0s - loss: 1.8795 - accuracy: 0.5982

Epoch 2: saving model to checkpoints/model_02.ckpt

2425/2425 [==============================] - 1707s 704ms/step - loss: 1.8795 - accuracy: 0.5982 - val_loss: 1.6312 - val_accuracy: 0.6364

Epoch 3/10

2425/2425 [==============================] - ETA: 0s - loss: 1.6922 - accuracy: 0.6256

Epoch 3: saving model to checkpoints/model_03.ckpt

2425/2425 [==============================] - 1703s 702ms/step - loss: 1.6922 - accuracy: 0.6256 - val_loss: 1.4751 - val_accuracy: 0.6606

Epoch 4/10

2425/2425 [==============================] - ETA: 0s - loss: 1.5477 - accuracy: 0.6477

Epoch 4: saving model to ch

In [11]:
import json
import os

# Directorio para guardar los datos
data_dir = "data"
os.makedirs(data_dir, exist_ok=True)

# Ruta del archivo para guardar la variable history como JSON
history_file = os.path.join(data_dir, "history.json")

# Convertir el diccionario 'history.history' a formato JSON y guardarlo en un archivo
with open(history_file, 'w') as f:
    json.dump(history.history, f)

In [12]:
import time
from tensorflow.keras.callbacks import Callback

# Obtener los tiempos por época y tiempo acumulado
times_per_epoch = time_callback.times
cumulative_time = [sum(times_per_epoch[:i+1]) for i in range(len(times_per_epoch))]

# Ruta del directorio para guardar los tiempos en un archivo de texto
time_dir = "time"
os.makedirs(time_dir, exist_ok=True)  # Crea la carpeta "time" si no existe

# Ruta del archivo para guardar los tiempos en un archivo de texto
time_file = os.path.join(time_dir, "tiempos.txt")

# Guardar la salida en el archivo de texto
with open(time_file, 'w') as f:
    print("Tiempos por época:", times_per_epoch, file=f)
    print("Tiempo acumulado:", cumulative_time, file=f)

# También puedes imprimir los resultados en la consola si lo deseas
print("Tiempos por época:", times_per_epoch)
print("Tiempo acumulado:", cumulative_time)

Tiempos por época: [1851.9821662902832, 1708.822984457016, 1704.590767621994, 1755.7161228656769, 1713.3261742591858, 1724.6881439685822, 1775.269733428955, 1737.0464487075806, 1774.6841344833374, 1786.9337086677551]

Tiempo acumulado: [1851.9821662902832, 3560.805150747299, 5265.395918369293, 7021.11204123497, 8734.438215494156, 10459.126359462738, 12234.396092891693, 13971.442541599274, 15746.126676082611, 17533.060384750366]



##  Métricas y parámetros para evaluar el rendimiento y la eficiencia durante el entrenamiento.

Una vez terminado el entrenamiento podemos visualizar y monitorear el rendimiento del modelo durante el entrenamiento, proporcionando métricas como pérdida, precisión, tasa de aprendizaje y eficiencia computacional.

import numpy as np
import matplotlib.pyplot as plt

# Obtener las métricas de pérdida, precisión y tasa de aprendizaje durante el entrenamiento
loss = history.history['loss']
accuracy = history.history['accuracy'] # Reemplaza 'accuracy' con 'weighted_accuracy' si estás utilizando esta métrica
learning_rate = np.array([learning_rate[step] for step in range(len(history.epoch))])

# Obtener la eficiencia computacional
time_per_epoch = history.epoch[-1] / len(history.epoch)  # Tiempo por época
efficiency = [time_per_epoch * (i + 1) for i in range(num_epochs)]

# Graficar la pérdida, precisión, tasa de aprendizaje y eficiencia
epochs = range(1, num_epochs + 1)

plt.figure(figsize=(12, 4))

plt.subplot(1, 4, 1)
plt.plot(epochs, loss, 'b', label='Pérdida')
plt.title('Pérdida durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Valor')
plt.legend()

plt.subplot(1, 4, 2)
plt.plot(epochs, accuracy, 'r', label='Precisión')
plt.title('Precisión durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Valor')
plt.legend()

plt.subplot(1, 4, 3)
plt.plot(epochs, learning_rate, 'g', label='Tasa de Aprendizaje')
plt.title('Tasa de Aprendizaje durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Valor')
plt.legend()

plt.subplot(1, 4, 4)
plt.plot(epochs, efficiency, 'm', label='Eficiencia')
plt.title('Eficiencia Computacional durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Tiempo (segundos)')
plt.legend()

plt.tight_layout()
plt.show()


Una vez finalizado el ajuste fino, puedes generar texto nuevamente utilizando la misma función generate(). Esta vez, el texto estará más cerca del estilo de wikihow y la longitud generada será similar a la longitud preestablecida en el conjunto de entrenamiento.

In [13]:
start = time.time()


output = gpt2_lm.generate("¿Como jugar videojuegos?", max_length=200)
print("\nGPT-2 output:")
print(output)

end = time.time()
print(f"TIEMPO TOTAL TRANSCURRIDO: {end - start:.2f}s")



GPT-2 output:

¿Como jugar videojuegos?

Preparar el juego:

Determina el objetivo del juego. Determina cuál será el orden que realizará el juego. Elige el número de jugadores para el juego.

Antes de que jugar videojuegos, debes tener en cuenta que el juego es el juego que determinará si quieres jugar otra persona, así como un equipo. Sin embargo,

TIEMPO TOTAL TRANSCURRIDO: 17.44s


## Acerca del Método de Muestreo

En KerasNLP, ofrecemos varios métodos de muestreo, como la búsqueda contrastiva, el muestreo Top-K y el muestreo de haz (beam sampling). Por defecto, nuestro `GPT2CausalLM` utiliza el muestreo Top-K, pero puedes elegir tu propio método de muestreo.

Al igual que con el optimizador y las funciones de activación, hay dos formas de especificar tu propio muestreador personalizado:

- Utilizar un identificador de cadena, como "greedy", si deseas utilizar la configuración predeterminada de esta forma.
- Pasar una instancia de `keras_nlp.samplers.Sampler`, si deseas utilizar una configuración personalizada de esta forma.

In [14]:
# Use a string identifier.
gpt2_lm.compile(sampler="top_k")
output = gpt2_lm.generate("Como escribir un ensayo", max_length=200)
print("\nGPT-2 output:")
print(output)

# Use a `Sampler` instance. `GreedySampler` tends to repeat itself,
greedy_sampler = keras_nlp.samplers.GreedySampler()
gpt2_lm.compile(sampler=greedy_sampler)

output = gpt2_lm.generate("Como escribir un ensayo", max_length=200)
print("\nGPT-2 output:")
print(output)



GPT-2 output:

Como escribir un ensayo de comparación y contraste?

Escribir tu propio ensayo de comparación y contraste:

Piensa en un tema. Escribe la oración y el enunciado. Escribe el cuerpo de la oración y el enunciado. Escribe una introducción. Escribe una conclusión. Escribe una conclusión. Escribe una conclusión.

Si tienes mucha experiencia, es probable que te hagan sentir un



GPT-2 output:

Como escribir un ensayo de comparación y contraste?

Escribir el ensayo:

Escribe una introducción. Escribe una conclusión. Escribe una conclusión informal. Escribe una conclusión más personal. Escribe una conclusión más cortés. Escribe una conclusión más abierta.

Empieza con una conclusión que tenga una conclusión general de tu tema o de tu situación


Para obtener más detalles sobre la clase `Sampler` de KerasNLP, puedes revisar el código [aquí](https://github.com/keras-team/keras-nlp/tree/master/keras_nlp/samplers).

## Guarda tu Modelo


*** Guarda el modelo: Utiliza el método model.save() de Keras para guardar todo el modelo, incluyendo la configuración y los pesos, en un archivo. Especifica una ruta y un nombre de archivo para guardar el modelo

import os
save_dir = "saved_model_h5"
os.makedirs(save_dir, exist_ok=True)
model.save("saved_model_h5/model.h5")

 Si estás trabajando principalmente con TensorFlow y planeas utilizar el modelo en otros proyectos o plataformas compatibles con SavedModel, entonces tf.saved_model.save() puede ser más conveniente.

import os
save_dir2 = "saved_model_tf"
os.makedirs(save_dir2, exist_ok=True)
tf.saved_model.save(gpt2_lm, saved_model_dir)

## Restaura tu modelo desde un checkpoint


preprocessor = keras_nlp.models.GPT2CausalLMPreprocessor.from_preset(
    "gpt2_base_en",
    sequence_length=128,
)
gpt2_lm = keras_nlp.models.GPT2CausalLM.from_preset(
    "gpt2_base_en", preprocessor=preprocessor
)

import tensorflow as tf
from tensorflow.keras import layers, models

# Crear el modelo con la misma arquitectura
preprocessor = keras_nlp.models.GPT2CausalLMPreprocessor.from_preset(
    "gpt2_base_en",
    sequence_length=128,
)
gpt2_lm = keras_nlp.models.GPT2CausalLM.from_preset(
    "gpt2_base_en", preprocessor=preprocessor
)

# Directorio donde se encuentran los pesos guardados
checkpoint_dir = "checkpoints/"

# Verificar si hay checkpoints existentes para cargar los pesos desde el último
checkpoint_files = [file for file in os.listdir(checkpoint_dir) if file.endswith('.ckpt.index')]
if checkpoint_files:
    latest_checkpoint = max(checkpoint_files)
    print("Ultimo checkpoint: ", latest_checkpoint)
    gpt2_lm.load_weights(os.path.join(checkpoint_dir, latest_checkpoint[:-6]))


loss,acc = gpt2_lm.evaluate(train_ds, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

start = time.time()

output = gpt2_lm.generate("Como conseguir diamante en minecraft", max_length=200)
print("\nGPT-2 output:")
print(output)

end = time.time()
print(f"TIEMPO TOTAL TRANSCURRIDO: {end - start:.2f}s")

import os

input_dir = '/kaggle/input'
files = [file for file in os.listdir(input_dir)]
print(files)

for file in files:
    file_path = os.path.join(input_dir, file)
    file_size_bytes = os.stat(file_path).st_size
    file_size_mb = file_size_bytes / 1048576  # 1 MB = 1048576 bytes
    print(f"File: {file}, Size: {file_size_mb:.2f} MB")
